In [ ]:
import sys
sys.path.insert(0, '.')
import os
import os.path as osp
import logging
import argparse
import math
import random
import time
import shutil
from tabulate import tabulate
from datetime import datetime
 
!pip install tensorboardX
from tensorboardX import SummaryWriter
 
import matplotlib.pyplot as plt
 
from tqdm import tqdm
import numpy as np
import cv2
import numpy as np
import collections
 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
 
from torchsummary import summary
from torchvision.datasets import Cityscapes
from torchvision import transforms

     |████████████████████████████████| 317kB 9.0MB/s 


In [ ]:
class ConvLayer(nn.Sequential):
    def __init__(self, in_channels, out_channels, kernel=3, stride=1, dropout=0.1):
        super().__init__()
        self.add_module('conv', nn.Conv2d(in_channels, out_channels, kernel_size=kernel,
                                          stride=stride, padding=kernel//2, bias = False))
        self.add_module('norm', nn.BatchNorm2d(out_channels))
        self.add_module('relu', nn.ReLU(inplace=True))

    def forward(self, x):
        return super().forward(x)

class TransitionUp(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

    def forward(self, x, skip, concat=True):
        out = F.interpolate(
                x,
                size=(skip.size(2), skip.size(3)),
                mode="bilinear",
                align_corners=True,
                            )
        if concat:                            
          out = torch.cat([out, skip], 1)
          
        return out

In [ ]:
class my_HarDBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, grmul, n_layers, keepBase=False):
        super().__init__()
        self.in_channels = in_channels
        self.growth_rate = growth_rate
        self.grmul = grmul
        self.links = []
        self.out_channels = 0
        self.keepBase = keepBase
        self.layers = nn.ModuleList([])
        for i in range(n_layers):
            out_ch, in_ch, el_links = self.get_links(i+1)
            self.links.append(el_links)
            self.layers.append(ConvLayer(in_ch, out_ch))
            if (i % 2 == 0) or (i == n_layers - 1):
                self.out_channels += out_ch

    def __out_ch(self, layer_id): 
        out_ch = self.growth_rate
        for i in range(1, int(math.log2(layer_id))+1):
            if layer_id % 2**i == 0:
                out_ch = out_ch*self.grmul
        return int(int(out_ch + 1) / 2) * 2

    def get_links(self, layer_id):
        in_ch = 0
        links_ = []
        for i in range(int(math.log2(layer_id))):
            diff = 2**i
            if (layer_id % diff == 0) and layer_id - diff > 0:
                in_ch += self.__out_ch(layer_id - diff)
                links_.append(layer_id - diff)
        if math.log2(layer_id).is_integer():
            in_ch += self.in_channels
            links_.append(0)
        return self.__out_ch(layer_id), in_ch, links_

    def forward(self, x):
        data = [x]
        for layer in range(len(self.layers)):
            layer_input = []
            for link in self.links[layer]:
                layer_input.append(data[link])
            in_ = layer_input[0] if len(layer_input) == 1 \
                else torch.cat(layer_input, dim=1)
            data.append(self.layers[layer](in_))
        t = len(data)
        out = []
        for i in range(t):
            if (i % 2 == 1) or (self.keepBase and i == 0) or (i == t-1):
                out.append(data[i])
        return torch.cat(out, dim=1)
    
    def get_out_ch(self):
        return self.out_channels

In [ ]:
class my_hardnet(nn.Module):
    def __init__(self, n_classes=19):
        super(my_hardnet, self).__init__()
        self.n_classes = n_classes

        first_ch  = [16,24,32,48]
        ch_list = [  64, 96, 160, 224, 320]
        grmul = 1.7
        gr       = [  10,16,18,24,32]
        n_layers = [   4, 4, 8, 8, 8]

        self.base = nn.ModuleList([
            ConvLayer(          3, first_ch[0], stride=2),
            ConvLayer(first_ch[0], first_ch[1], stride=1),
            ConvLayer(first_ch[1], first_ch[2], stride=2),
            ConvLayer(first_ch[2], first_ch[3], stride=1)
        ])

        blks_count = len(n_layers)
        ch = first_ch[3]
        skip_connection_channel_counts = []
        self.shortcut_layers = []
        for i in range(blks_count):
            block = my_HarDBlock(ch, gr[i], grmul, n_layers[i])
            ch = block.get_out_ch()
            skip_connection_channel_counts.append(ch)
            
            self.base.append(block)
            if i < blks_count-1:
                self.shortcut_layers.append(len(self.base)-1)
            
            self.base.append(ConvLayer(ch, ch_list[i], kernel=1))
            ch = ch_list[i]
            if (i < blks_count - 1):
                self.base.append(nn.AvgPool2d(kernel_size=2, stride=2))

        self.conv1x1_up = nn.ModuleList()
        self.transUp = nn.ModuleList()
        self.denseUp = nn.ModuleList()

        blks_count -= 1
        self.blks_count = blks_count 

        for i in range(blks_count - 1, -1, -1):
            self.transUp.append(TransitionUp(ch, ch))
            ch = ch + skip_connection_channel_counts[i]
            self.conv1x1_up.append(ConvLayer(ch, ch//2, kernel=1))
            ch = ch//2
            blk = my_HarDBlock(ch, gr[i], grmul, n_layers[i])
            self.denseUp.append(blk)
            ch = blk.get_out_ch()

        self.finalConv = nn.Conv2d(ch, n_classes, kernel_size=1, 
                                   stride=1, padding=0, bias=True)

    def forward(self, x):
        size_in = x.size()
        out = x
        save_shortcut = []
        for i in range(len(self.base)):
            out = self.base[i](out)
            if i in self.shortcut_layers:
                save_shortcut.append(out)
        
        for i in range(self.blks_count):
            skip = save_shortcut.pop()
            out = self.transUp[i](out, skip, True)
            out = self.conv1x1_up[i](out)
            out = self.denseUp[i](out)

        out = self.finalConv(out)
        
        out = F.interpolate(out,
                            size=(size_in[2], size_in[3]),
                            mode="bilinear",
                            align_corners=True)
        return out

In [ ]:
!git clone https://github.com/PingoLH/FCHarDNet.git
!mv FCHarDNet/ptsemseg ./

Cloning into 'FCHarDNet'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 120 (delta 0), reused 2 (delta 0), pack-reused 117
Receiving objects: 100% (120/120), 9.09 MiB | 13.11 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
import yaml
from torch.utils import data

from ptsemseg.models import get_model
from ptsemseg.loss import get_loss_function
from ptsemseg.loader import get_loader
from ptsemseg.utils import get_logger
from ptsemseg.metrics import runningScore, averageMeter
from ptsemseg.augmentations import get_composed_augmentations
from ptsemseg.schedulers import get_scheduler
from ptsemseg.optimizers import get_optimizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('FCHarDNet/configs/hardnet.yml') as cf:
    cfg = yaml.load(cf)
    cfg['data'].update({
        'path':'/content/drive/MyDrive/RnD/datasets/',
        'sbd_path': '/content/drive/MyDrive/RnD/datasets/',
    })
    cfg['training'].update({
        'train_iters': 24000,
        'batch_size': 6,
        'optimizer': {
            'name': 'adam'
        }
    })
    cfg['training']['lr_schedule'].update({
        'max_iter': 120000
    })

In [ ]:
# Setup seeds
torch.manual_seed(cfg.get("seed", 1337))
torch.cuda.manual_seed(cfg.get("seed", 1337))
np.random.seed(cfg.get("seed", 1337))
random.seed(cfg.get("seed", 1337))

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Setup Augmentations
augmentations = cfg["training"].get("augmentations", None)
data_aug = get_composed_augmentations(augmentations)

# Setup Dataloader
data_loader = get_loader(cfg["data"]["dataset"])
data_path = cfg["data"]["path"]

t_loader = data_loader(
    data_path,
    is_transform=True,
    split=cfg["data"]["train_split"],
    img_size=(cfg["data"]["img_rows"], cfg["data"]["img_cols"]),
    augmentations=data_aug,
)
n_classes = t_loader.n_classes
trainloader = data.DataLoader(
    t_loader,
    batch_size=cfg["training"]["batch_size"],
    num_workers=cfg["training"]["n_workers"],
    shuffle=True,
)

v_loader = data_loader(
    data_path,
    is_transform=True,
    split=cfg["data"]["val_split"],
    img_size=(1024,2048),
)
valloader = data.DataLoader(
    v_loader, batch_size=cfg["training"]["batch_size"], num_workers=cfg["training"]["n_workers"]
)

Found 2975 train images
Found 500 val images


In [ ]:
running_metrics_val = runningScore(n_classes)
model = my_hardnet().to(device)
total_params = sum(p.numel() for p in model.parameters())
print( 'Parameters:',total_params )
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight)

model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.apply(weights_init)
pass

Parameters: 4119257


In [ ]:
optimizer_cls = get_optimizer(cfg)
optimizer_params = {k: v for k, v in cfg["training"]["optimizer"].items() if k != "name"}
optimizer = optimizer_cls(model.parameters(), **optimizer_params)
# scheduler = get_scheduler(optimizer, cfg["training"]["lr_schedule"])
loss_fn = get_loss_function(cfg)

print("Using optimizer {}".format(optimizer))
print("Using loss {}".format(loss_fn))

val_loss_meter = averageMeter()
time_meter = averageMeter()

Using optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 0.001
    lr: 0.001
    weight_decay: 0
)
Using loss functools.partial(<function bootstrapped_cross_entropy2d at 0x7fee516d8f80>, min_K=4096, loss_th=0.3, size_average=True)


In [ ]:
start_iter = 0
best_iou = -100.0
i = start_iter
flag = True
loss_all = 0
loss_n = 0

In [ ]:
logdir = os.path.join("/content/drive/MyDrive/RnD/runs/adam", str(datetime.fromtimestamp(int(time.time()))))
writer = SummaryWriter(log_dir=logdir)

print("RUNDIR: {}".format(logdir))
# shutil.copy(args.config, logdir)

logger = get_logger(logdir)
logger.info("Let the games begin")

INFO:ptsemseg:Let the games begin


RUNDIR: /content/drive/MyDrive/RnD/runs/adam/2021-02-27 07:47:40


In [ ]:
!ls /content/drive/MyDrive/RnD/runs/adam

'2021-02-26 19:59:12'


In [ ]:
loaded = torch.load('/content/drive/MyDrive/RnD/runs/adam/2021-02-26 19:59:12/hardnet_cityscapes_checkpoint.pkl')
best_iou_arrc = torch.load('/content/drive/MyDrive/RnD/runs/adam/2021-02-26 19:59:12/hardnet_cityscapes_best_model.pkl')
model_state = loaded.get('model_state')

optimizer_state = loaded.get('optimizer_state')
# scheduler_state = loaded.get('scheduler_state')
start_iter = loaded.get('epoch')

best_iou = best_iou_arrc.get('best_iou')
i = start_iter
flag = True
loss_all = 0
loss_n = 0

model.load_state_dict(model_state)
optimizer.load_state_dict(optimizer_state)
# scheduler.load_state_dict(scheduler_state)

In [ ]:
i, cfg["training"]["train_iters"], best_iou

(12000, 24000, 0.4852946193149939)

In [ ]:
while i <= cfg["training"]["train_iters"] and flag:
    for (images, labels, _) in trainloader:
        i += 1
        start_ts = time.time()
        model.train()
        images = images.to(device)
        labels = labels.to(device)
 
        outputs = model(images)
 
        loss = loss_fn(input=outputs, target=labels)
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        # scheduler.step()
        
        # c_lr = scheduler.get_lr()
 
        time_meter.update(time.time() - start_ts)
        loss_all += loss.item()
        loss_n += 1
        
        if (i + 1) % cfg["training"]["print_interval"] == 0:
            fmt_str = "Iter [{:d}/{:d}]  Loss: {:.4f}  Time/Image: {:.4f}"#  lr={:.6f}
            print_str = fmt_str.format(
                i + 1,
                cfg["training"]["train_iters"],
                loss_all / loss_n,
                time_meter.avg / cfg["training"]["batch_size"],
                # c_lr[0],
            )
            
 
            print(print_str)
            logger.info(print_str)
            writer.add_scalar("loss/train_loss", loss.item(), i + 1)
            time_meter.reset()
 
        if (i + 1) % cfg["training"]["val_interval"] == 0 or (i + 1) == cfg["training"][
            "train_iters"
        ]:
            torch.cuda.empty_cache()
            model.eval()
            loss_all = 0
            loss_n = 0
            with torch.no_grad():
                for i_val, (images_val, labels_val, _) in tqdm(enumerate(valloader)):
                    images_val = images_val.to(device)
                    labels_val = labels_val.to(device)
 
                    outputs = model(images_val)
                    val_loss = loss_fn(input=outputs, target=labels_val)
 
                    pred = outputs.data.max(1)[1].cpu().numpy()
                    gt = labels_val.data.cpu().numpy()
 
                    running_metrics_val.update(gt, pred)
                    val_loss_meter.update(val_loss.item())
 
            writer.add_scalar("loss/val_loss", val_loss_meter.avg, i + 1)
            logger.info("Iter %d Val Loss: %.4f" % (i + 1, val_loss_meter.avg))
 
            score, class_iou = running_metrics_val.get_scores()
            for k, v in score.items():
                print(k, v)
                logger.info("{}: {}".format(k, v))
                writer.add_scalar("val_metrics/{}".format(k), v, i + 1)
 
            for k, v in class_iou.items():
                logger.info("{}: {}".format(k, v))
                writer.add_scalar("val_metrics/cls_{}".format(k), v, i + 1)
 
            val_loss_meter.reset()
            running_metrics_val.reset()
            
            state = {
                    "epoch": i + 1,
                    "model_state": model.state_dict(),
                    "optimizer_state": optimizer.state_dict(),
                    # "scheduler_state": scheduler.state_dict(),
            }
            save_path = os.path.join(
                writer.file_writer.get_logdir(),
                "{}_{}_checkpoint.pkl".format(cfg["model"]["arch"], cfg["data"]["dataset"]),
            )
            torch.save(state, save_path)
 
            if score["Mean IoU : \t"] >= best_iou:
                best_iou = score["Mean IoU : \t"]
                state = {
                    "epoch": i + 1,
                    "model_state": model.state_dict(),
                    "best_iou": best_iou,
                }
                save_path = os.path.join(
                    writer.file_writer.get_logdir(),
                    "{}_{}_best_model.pkl".format(cfg["model"]["arch"], cfg["data"]["dataset"]),
                )
                torch.save(state, save_path)
            torch.cuda.empty_cache()
 
        if (i + 1) == cfg["training"]["train_iters"]:
            flag = False
            break

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
INFO:ptsemseg:Iter [12010/24000]  Loss: 1.0709  Time/Image: 0.1464  lr=0.000909


Iter [12010/24000]  Loss: 1.0709  Time/Image: 0.1464  lr=0.000909


INFO:ptsemseg:Iter [12020/24000]  Loss: 1.0809  Time/Image: 0.1444  lr=0.000909


Iter [12020/24000]  Loss: 1.0809  Time/Image: 0.1444  lr=0.000909


INFO:ptsemseg:Iter [12030/24000]  Loss: 1.0634  Time/Image: 0.1426  lr=0.000909


...

Iter [23990/24000]  Loss: 0.9605  Time/Image: 0.1642  lr=0.000818


INFO:ptsemseg:Iter [24000/24000]  Loss: 0.9600  Time/Image: 0.1654  lr=0.000818


Iter [24000/24000]  Loss: 0.9600  Time/Image: 0.1654  lr=0.000818


84it [03:20,  2.39s/it]
INFO:ptsemseg:Iter 24000 Val Loss: 0.9968
INFO:ptsemseg:Overall Acc: 	: 0.9290885987796044
INFO:ptsemseg:Mean Acc : 	: 0.6613456771470932
INFO:ptsemseg:FreqW Acc : 	: 0.8754134502975653
INFO:ptsemseg:Mean IoU : 	: 0.5628809774448534
INFO:ptsemseg:0: 0.96479338409809
INFO:ptsemseg:1: 0.7391822706254285
INFO:ptsemseg:2: 0.8746804413142809
INFO:ptsemseg:3: 0.28554161966055486
INFO:ptsemseg:4: 0.43992511319558636
INFO:ptsemseg:5: 0.4477680538540516
INFO:ptsemseg:6: 0.44531852713440156
INFO:ptsemseg:7: 0.5868226183116098
INFO:ptsemseg:8: 0.8858165134678683
INFO:ptsemseg:9: 0.5012465860918411
INFO:ptsemseg:10: 0.9175465161960255
INFO:ptsemseg:11: 0.6618482415769881
INFO:ptsemseg:12: 0.27884679876805274
INFO:ptsemseg:13: 0.8956801023898054
INFO:ptsemseg:14: 0.19563375955378173
INFO:ptsemseg:15: 0.5085003798530466
INFO:ptsemseg:16: 0.2907954128662374
INFO:ptsemseg:17: 0.15337719134302696
INFO:ptsemseg:18: 0.6214150411515355


Overall Acc: 	 0.9290885987796044
Mean Acc : 	 0.6613456771470932
FreqW Acc : 	 0.8754134502975653
Mean IoU : 	 0.5628809774448534
